# TP 3 Machine Learning
**Fecha y hora de entrega:** 30/05/2022 18:00

*Para más información visitar [el TP en la wiki oficial de la materia](https://github.com/ucseml-team/machine-learning-course/wiki/TP3_2022)*

# **Fashion-MNIST Dataset**

Se trata de un conjunto de datos de 60.000 imágenes en escala de grises de 28x28 de 10 categorías de moda, junto con un conjunto de prueba de 10.000 imágenes. Este conjunto de datos puede utilizarse como reemplazo de MNIST.

<img src="https://tensorflow.org/images/fashion-mnist-sprite.png"></img>


**Las clases son:**

Label   | Description
--------|------------------
0       | T-shirt/top - Remera/Top
1       | Trouser - Pantalón
2       | Pullover - Jersey
3       | Dress - Vestido
4       | Coat - Abrigo
5       | Sandal - Sandalia
6       | Shirt - Camisa
7       | Sneaker - Zapatilla
8       | Bag - Bolso
9       | Ankle boot - Botas


**Retorna**

Tupla de matrices NumPy: (x_train, y_train), (x_test, y_test):

**x_train**: matriz NumPy de uint8 de datos de imagen en escala de grises con formas (60000, 28, 28), que contiene los datos de train.

**y_train**: matriz NumPy uint8 de etiquetas (enteros en el rango 0-9) con forma (60000,) para los datos de train.

**x_test**: uint8 NumPy array de datos de imagen en escala de grises con forma (10000, 28, 28), que contiene los datos de test.

**y_test**: matriz NumPy uint8 de etiquetas (enteros en el rango 0-9) con forma (10000,) para los datos de test.

## **Cargando al Fashion-MNIST dataset**

> **load_data** function


```
keras.datasets.fashion_mnist.load_data()
```

---


*Para más información acerca del dataset visitar a [la página oficial de Fashion MNIST](https://keras.io/api/datasets/fashion_mnist/)*

In [ ]:
# !pip install -r requirements.txt

In [ ]:
# de python, para especificar rutas de archivos y directorios
from pathlib import Path

# lib para trabajar con arrays
import numpy as np

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para construir y entrenar redes neuronales, y que además tiene 
# utilidades para leer sets de imágenes

# TensorFlow y tf.keras
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

import pandas as pd

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

## Train and Test

In [ ]:
# lo vamos a estar usando seguido
CLOTHES = "Remera/Top", "Pantalón", "Jersey", "Vestido", "Abrigo", "Sandalia", "Camisa", "Zapatilla", "Bolso", "Botas"

# (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Análisis exploratorio sobre el conjunto de datos

## Volumetría de los datos

In [ ]:
# Ver las dimensiones del dataset
assert train_images.shape == (60000, 28, 28)
assert test_images.shape == (10000, 28, 28)
assert train_labels.shape == (60000,)
assert test_labels.shape == (10000,)

In [ ]:
# Mostrar la cantidad de datos en train
data_shape = [('train_images', len(train_images), train_images.shape),
              ('train_labels', len(train_labels), train_labels.shape),
              ('test_images', len(test_images), test_images.shape),
              ('test_labels', len(test_labels), test_labels.shape)]
pd.DataFrame(data_shape, columns=['Dataset Name', 'Cantidad de datos', 'Shape'])

In [ ]:
train_labels

## Estructura y tipo de las imágenes

In [ ]:
def print_dataset_images(dimensions, images, labels, isColorbar=False):
  plt.figure(figsize=(10,10))
  for i in range(dimensions*dimensions):
      plt.subplot(dimensions,dimensions,i+1)
      plt.xticks([])
      plt.yticks([])
      plt.imshow(images[i])
      if isColorbar: plt.colorbar()
      plt.grid(False)
      plt.title("La clase es: {} \n {}".format(labels[i], CLOTHES[labels[i]]))
  plt.subplots_adjust(top=1.1, right=1)
  plt.show()

In [ ]:
print_dataset_images(5, train_images, train_labels, True)

In [ ]:
print_dataset_images(5, train_images, train_labels)

## Distribución de la variable a predecir

Se puede observar que en ambos datasets, tanto train como test, los datos se encuentran completamente balanceados

In [ ]:
lista_train = list(range(10))
for label in train_labels: 
    lista_train[label] += 1 
plt.title("distibucion prendas en train")
plt.pie(lista_train, labels=CLOTHES, autopct="%0.1f %%");

In [ ]:
lista_test = list(range(10))
for label in test_labels: 
    lista_test[label] += 1 
plt.title("distibucion prendas en test")
plt.pie(lista_test, labels=CLOTHES, autopct="%0.1f %%");

# Preprocesado del dataset a trabajar

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
print_dataset_images(5, train_images, train_labels, True)

# Acá empieza la magiaa 